# Primo esperimento deep learning

* Architettura MLP con un solo strato nascosto da 300 unità
* Ottimizzazione tramite SGD con learning rate a 0.01 no decay e minibatch da 500 sample
* GridSearch per cercare le migliori funzion di attivazione\perdita:
    * attivazioni: softmax, softplus, tanh, relu, sigmoid
    * perdita: mean squared error, binary cross entropy, hinge loss
* L'attivazione dell'unità di uscita la fisso con una sigmoidale
* Inizializzatore dei pesi: glorot uniforme
* Numero di epoche: 10
Il training set e il test set non vengono alterati.
### Librerie utilizzate
* keras:2.2.0
* scikit-learn:0.19.1
* pandas:0.23.0
* numpy:1.14.5
* theano:1.0.2

In [1]:
#Per prima cosa importiamo tutto ciò di cui abbiamo bisogno
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
import keras.backend
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from keras.optimizers import SGD
from numpy.random import seed
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
from sklearn.metrics import make_scorer
from sklearn.metrics import auc
from sklearn.metrics import precision_recall_curve
from sklearn.externals import joblib
from sklearn.model_selection import cross_validate
from sklearn.metrics import average_precision_score
import theano.tensor as T
import copy

Using Theano backend.
WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


Definiamo la griglia dei parametri da ricercare e il numero di feature per ogni esempio e settiamo il seed numpy per i numeri casuali

In [2]:
def hingesig(y_true, y_pred):
    transform_y_true = T.switch(T.eq(y_true,0), -1, y_true)
    clipped_y_pred = T.clip(y_pred, 1e-20, 0.9999999)
    logit = (T.log2(clipped_y_pred) - T.log2(T.sub(1., clipped_y_pred)))
    return T.mean(T.maximum(1. - transform_y_true * logit, 0.), axis=-1)

params = {
        'loss': ['binary_crossentropy', 'mean_absolute_error', hingesig],
        'activation': ['softmax', 'softplus', 'tanh', 'relu', 'sigmoid']
    }

feature_per_example=26
batch_size = 500
my_seed = 2024 #imposto un valore di seed da dare a tutti i generatori
seed(my_seed) #seed numpy

Keras per funzionare con scikit mette a disposizione la classe `KerasClassifier` che fa da wrapper. Ha bisogno di una funzione che crea e compila il modello

In [3]:
def create_model(loss='binary_crossentropy', activation='tanh'):
    model = Sequential()
    initializer = keras.initializers.glorot_uniform(seed=my_seed)
    model.add(Dense(
            300, 
            input_dim=feature_per_example, 
            kernel_initializer=initializer,
            activation=activation)
             )
    model.add(Dense(
            1,
            kernel_initializer=initializer,
            activation='sigmoid'
    ))
    optimizer = SGD(lr=0.01, decay=0, momentum=0, nesterov=False)
    model.compile(loss=loss, optimizer=optimizer)
    return model
create_model()

Carichiamo i dati tramite pandas e creiamo il vettore di etichette

In [4]:
base_data_folder = "/home/alessio/dati/"
train_set_filename = base_data_folder + "Mendelian.train.tsv"
test_set_filename = base_data_folder + "Mendelian.test.tsv"

train_X = pd.read_csv(train_set_filename, sep='\t').values
test_X = pd.read_csv(test_set_filename, sep='\t').values
#creiamo le label, nel train set i primi 356 esempi sono positivi, nel test i primi 40 sono positivi
n_positives = 356
n_negatives = train_X.shape[0] - n_positives
train_y = np.concatenate((
    np.ones(n_positives, dtype=np.int32),
    np.zeros(n_negatives, dtype=np.int32)
))
n_positives = 40
n_negatives = test_X.shape[0] - n_positives
test_y = np.concatenate((
    np.ones(n_positives, dtype=np.int32),
    np.zeros(n_negatives, dtype=np.int32)
))

Creiamo la funzione di scoring, calcolando l'AUC per la precision recall curve specifichiamo `reorder=False` perchè non è una curva "ascendente", come nel caso della curva ROC .

In [7]:
def prc_score(y_true, y_pred):
    precision, recall, _ = precision_recall_curve(y_true=y_true, probas_pred=y_pred)
    return auc(x=recall, y=precision)

scoring = {
    'AU_PRC': make_scorer(prc_score, needs_threshold=True),
    'AU_ROC': make_scorer(roc_auc_score, needs_threshold=True),
    'AVG_PREC': make_scorer(average_precision_score, needs_threshold=True)
}

In [ ]:
model = KerasClassifier(build_fn = create_model, epochs = 10, batch_size=batch_size, verbose=1, shuffle=False)
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=my_seed)
grid_search = GridSearchCV(estimator=model, 
                           param_grid=params, 
                           scoring=scoring,
                           refit='AVG_PREC',
                           cv=kfold, 
                           return_train_score=True,
                           n_jobs=6)
grid_search.fit(train_X, train_y)
print(grid_search.best_params_)
#saving best model
joblib.dump(grid_search.best_estimator_,'best_estimator.pkl')
#saving cv_results_
cv_results = pd.DataFrame.from_dict(grid_search.cv_results_)
cv_results.to_csv("cv_results.csv", index=False)